In [ ]:
#import sys
#!{sys.executable} -m pip install ortools 

import pandas as pd
import numpy as np
import time
import random
from ortools.linear_solver import pywraplp

In [ ]:
InputData = pd.read_excel("Nutrition Info.xlsx", sep="\t")
InputData

In [ ]:
PrefData = pd.read_excel("Nutrition Preferences.xlsx", sep="\t")
PrefData

In [ ]:
# Determine the start time
StartTime = time.process_time()

# Define our Linear Program
Solver = pywraplp.Solver('Solver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Define the Preference Coefficient P[i,j] for Shiva, for Ingredient i used for Meal j
n=19
m=5

P = np.zeros(shape=(n, m), dtype=int)
blocks = ["MON", "TUE", "WED", "THU", "FRI"]

for j in range(m):
    for i in range(n):
        P[i,j] = PrefData["Shiva"][i]
        
# Define the variable X[i,j], which will equal the number of times Ingredient i is used for Meal j
# No ingredient can be used more than twice.
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = Solver.IntVar(0, 2, 'X[%d, %d]' % (i,j))
        
# Set up our Happiness Function for SHIVA, which maximizes the total number of Happiness Points
HappinessFunction = Solver.Sum(P[i,j]*X[i,j] for i in range(n) for j in range(m))
Solver.Maximize(HappinessFunction)

# Include our first constraint: Each meal must have at least 700 calories
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Calories"][i]*X[i,j] for i in range(n)]) >=
               InputData["Calories"][19] )
                 
# Include our second constraint: Each meal must have at most 24 grams of fat
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Fat"][i]*X[i,j] for i in range(n)]) <=
               InputData["Fat"][19] )
                 
# Include our third constraint: Each meal must have at most 3 grams of sodium
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Sodium"][i]*X[i,j] for i in range(n)]) <=
               InputData["Sodium"][19] )

# Include our fourth constraint: Each meal must have at least 90 grams of carbs
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Carbs"][i]*X[i,j] for i in range(n)]) >=
               1*InputData["Carbs"][19] )
    
# Include our fifth constraint: Each meal must have at least 8 grams of dietary fiber
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Fiber"][i]*X[i,j] for i in range(n)]) >=
               1*InputData["Fiber"][19] )

# Include our sixth constraint: Each meal must have at least 17 grams of protein
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Protein"][i]*X[i,j] for i in range(n)]) >=
               1*InputData["Protein"][19] )

# Include our seventh constraints: no beef, no ham, no salami,
# and also, at most one serving of chicken in any two day stretch

Solver.Add(X[11,0]+X[12,0]+X[15,0]==0)
Solver.Add(X[11,1]+X[12,1]+X[15,1]==0)
Solver.Add(X[11,2]+X[12,2]+X[15,2]==0)
Solver.Add(X[11,3]+X[12,3]+X[15,3]==0)
Solver.Add(X[11,4]+X[12,4]+X[15,4]==0)
Solver.Add(X[13,0]+X[14,0]+X[13,1]+X[14,1]<=1)
Solver.Add(X[13,1]+X[14,1]+X[13,2]+X[14,2]<=1)
Solver.Add(X[13,2]+X[14,2]+X[13,3]+X[14,3]<=1)
Solver.Add(X[13,3]+X[14,3]+X[13,4]+X[14,4]<=1)



# Include our final constraint: Each meal must have at least 6 ingredients and at most 10 
for j in range(m):
    Solver.Add(Solver.Sum([X[i,j] for i in range(n)]) >= 6)
    Solver.Add(Solver.Sum([X[i,j] for i in range(n)]) <= 10)

# Solve the Integer Linear program
Output = Solver.Solve()
TotalPoints = round(Solver.Objective().Value())

# Determine the total time of running the program.
TotalTime = round(time.process_time() - StartTime, 4)

# Output one of the possible optimal solutions.
print("Python returns a solution with", TotalPoints, "Total Happiness Points in",
      TotalTime, "seconds")
print()
print("Shiva's menu")
for j in range(m):
    print()
    for i in range(n):
        if X[i,j].solution_value()==1:
            print("The meal on", blocks[j], "will have one serving of", 
                  InputData["Ingredient"][i], "with score", P[i,j])
        if X[i,j].solution_value()==2:
            print("The meal on", blocks[j], "will have two servings of", 
                  InputData["Ingredient"][i], "with score", P[i,j])


In [ ]:
# Determine the start time
StartTime = time.process_time()

# Define our Linear Program
Solver = pywraplp.Solver('Solver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Define the Preference Coefficient P[i,j] for DP, for Ingredient i used for Meal j
n=19
m=5

P = np.zeros(shape=(n, m), dtype=int)
blocks = ["MON", "TUE", "WED", "THU", "FRI"]

for j in range(m):
    for i in range(n):
        P[i,j] = PrefData["DP"][i]
        
# Define the variable X[i,j], which will equal the number of times Ingredient i is used for Meal j
# No ingredient can be used more than twice.
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = Solver.IntVar(0, 2, 'X[%d, %d]' % (i,j))
        
# Set up our Happiness Function for DP, which maximizes the total number of Happiness Points
HappinessFunction = Solver.Sum(P[i,j]*X[i,j] for i in range(n) for j in range(m))
Solver.Maximize(HappinessFunction)

# Include our first constraint: Each meal must have at least 700 calories
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Calories"][i]*X[i,j] for i in range(n)]) >=
               InputData["Calories"][19] )
                 
# Include our second constraint: Each meal must have at most 24 grams of fat
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Fat"][i]*X[i,j] for i in range(n)]) <=
               InputData["Fat"][19] )
                 
# Include our third constraint: Each meal must have at most 3 grams of sodium
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Sodium"][i]*X[i,j] for i in range(n)]) <=
               InputData["Sodium"][19] )

# Include our fourth constraint: Each meal must have at least 90 grams of carbs
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Carbs"][i]*X[i,j] for i in range(n)]) >=
               1*InputData["Carbs"][19] )
    
# Include our fifth constraint: Each meal must have at least 8 grams of dietary fiber
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Fiber"][i]*X[i,j] for i in range(n)]) >=
               1*InputData["Fiber"][19] )

# Include our sixth constraint: Each meal must have at least 17 grams of protein
for j in range(m):
    Solver.Add(Solver.Sum([InputData["Protein"][i]*X[i,j] for i in range(n)]) >=
               1*InputData["Protein"][19] )

# Include our seventh constraints: one serving of fries on Wednesday and Friday, 
# no spinach and cabbage together, and no chicken on Fridays
for j in range(m):
    Solver.Add(X[17,2]==1)
    Solver.Add(X[17,4]==1)
    Solver.Add(X[1,j]+X[4,j]<=1)
    Solver.Add(X[13,4]+X[14,4]==0)
    
# Include our final constraint: Each meal must have at least 6 ingredients and at most 10.
for j in range(m):
    Solver.Add(Solver.Sum([X[i,j] for i in range(n)]) >= 6)
    Solver.Add(Solver.Sum([X[i,j] for i in range(n)]) <= 10)
    

# Solve the Integer Linear program
Output = Solver.Solve()
TotalPoints = round(Solver.Objective().Value())

# Determine the total time of running the program.
TotalTime = round(time.process_time() - StartTime, 4)

# Output one of the possible optimal solutions.
print("Python returns a solution with", TotalPoints, "Total Happiness Points in",
      TotalTime, "seconds")
print()
print("DP's menu")
for j in range(m):
    print()
    for i in range(n):
        if X[i,j].solution_value()==1:
            print("The meal on", blocks[j], "will have one serving of", 
                  InputData["Ingredient"][i], "with score", P[i,j])
        if X[i,j].solution_value()==2:
            print("The meal on", blocks[j], "will have two servings of", 
                  InputData["Ingredient"][i], "with score", P[i,j])